Simple UI demo, extract data and performs inference for an area by given coordinates
Tested in Google colab to work with Google Earth Engine

In [ ]:
!pip install geopandas
import ee
import geopandas as gpd
import numpy as np
from shapely.ops import split
from shapely.geometry import MultiPolygon, Polygon, LineString
import threading
import concurrent.futures
import joblib
from sklearn.ensemble import RandomForestRegressor

In [ ]:
ee.Authenticate()
ee.Initialize()

In [ ]:
## extracting hila polygon
## splits the area given by coordinates to 16x16m areas.
# Due to projection issues, precision is not queranteed
# as orientation is differs between projections.
# should average over when using larger areas.

def extract_hila(coords):
  poly = gpd.GeoDataFrame([Polygon(coords)], columns={"geometry"}, crs="EPSG:4326")
  poly = poly.to_crs(epsg=2039)
  G = np.random.choice(poly.geometry.values)
  hilas = split_polygon(G,shape='square',thresh=0.5,side_length=16)
  
  res = gpd.GeoDataFrame(geometry=hilas, crs="EPSG:2039")
  res = res.to_crs(epsg=4326)
  return res


def get_squares_from_rect(RectangularPolygon, side_length=0.0025):
    """
    Divide a Rectangle (Shapely Polygon) into squares of equal area.

    `side_length` : required side of square

    """
    rect_coords = np.array(RectangularPolygon.boundary.coords.xy)
    y_list = rect_coords[1]
    x_list = rect_coords[0]
    y1 = min(y_list)
    y2 = max(y_list)
    x1 = min(x_list)
    x2 = max(x_list)
    width = x2 - x1
    height = y2 - y1

    xcells = int(np.round(width / side_length))
    ycells = int(np.round(height / side_length))

    yindices = np.linspace(y1, y2, ycells + 1)
    xindices = np.linspace(x1, x2, xcells + 1)
    horizontal_splitters = [
        LineString([(x, yindices[0]), (x, yindices[-1])]) for x in xindices
    ]
    vertical_splitters = [
        LineString([(xindices[0], y), (xindices[-1], y)]) for y in yindices
    ]
    result = RectangularPolygon
    for splitter in vertical_splitters:
        result = MultiPolygon(split(result, splitter))
    for splitter in horizontal_splitters:
        result = MultiPolygon(split(result, splitter))
    square_polygons = list(result)

    return square_polygons


def split_polygon(G, side_length=0.025, shape="square", thresh=0.9):
    """
    Using a rectangular envelope around `G`, creates a mesh of squares of required length.
    
    Removes non-intersecting polygons. 
            

    Args:
    
    - `thresh` : Range - [0,1]

        This controls - the number of smaller polygons at the boundaries.
        
        A thresh == 1 will only create (or retain) smaller polygons that are 
        completely enclosed (area of intersection=area of smaller polygon) 
        by the original Geometry - `G`.
        
        A thresh == 0 will create (or retain) smaller polygons that 
        have a non-zero intersection (area of intersection>0) with the
        original geometry - `G` 

    - `side_length` : Range - (0,infinity)
        side_length must be such that the resultant geometries are smaller 
        than the original geometry - `G`, for a useful result.

        side_length should be >0 (non-zero positive)

    - `shape` : {square/rhombus}
        Desired shape of subset geometries. 


    """
    assert side_length>0, "side_length must be a float>0"
    Rectangle    = G.envelope
    squares      = get_squares_from_rect(Rectangle, side_length=side_length)
    SquareGeoDF  = gpd.GeoDataFrame(squares).rename(columns={0: "geometry"})
    Geoms        = SquareGeoDF[SquareGeoDF.intersects(G)].geometry.values
    if shape == "square":
        geoms = [g for g in Geoms if ((g.intersection(G)).area / g.area) >= thresh]
    return geoms

In [ ]:
## Sentinel 2 satelite data processing
# Recommembed settings for satellite image filtering
CLOUD_FILTER = 60
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100

def get_s2_img(aoi, start_date='2021-07-01', end_date='2021-08-01'):    
    s2_sr_cld_col = get_s2_sr_cld_col(aoi, start_date, end_date)
    return (s2_sr_cld_col.map(add_cld_shdw_mask)
                .map(apply_cld_shdw_mask)
                .median()) 
    
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))


def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))


def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectancemethods bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)


def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)


#extract bands from sentinel data
def hila_bands(x, band):
  aoi = ee.Geometry.Polygon(x)
  img = get_s2_img(aoi)

  mean_band = img.select(band).reduceRegion(**{
    "reducer": ee.Reducer.mean(),
    "geometry": aoi,
    "scale": 10,  # A nominal scale in meters of the projection to work in
  })

  std_band = img.select(band).reduceRegion(**{
    "reducer": ee.Reducer.stdDev(),
    "geometry": aoi,
    "scale": 10,  # A nominal scale in meters of the projection to work in
  })

  max_band = img.select(band).reduceRegion(**{
    "reducer": ee.Reducer.max(),
    "geometry": aoi,
    "scale": 10,  # A nominal scale in meters of the projection to work in
  })

  min_band = img.select(band).reduceRegion(**{
    "reducer": ee.Reducer.min(),
    "geometry": aoi,
    "scale": 10,  # A nominal scale in meters of the projection to work in
  })

  return [mean_band.getNumber(band), std_band.getNumber(band), max_band.getNumber(band), min_band.getNumber(band)]

In [ ]:
## processing hila grids
#extract band values for the hila-grids
#Collect Numbers to List, evaluate once to save time in getInfo() calls
def extract_bands(coords, band):
  ln = len(coords)
  chunk = 500
  res = []

  # due to user memory limit, need to work in 1K chunks of data
  for i in range(0, ln, chunk):
    lim = i+chunk
    if lim > ln:
      lim = ln

    tmp = coords[i:lim].apply(lambda x: hila_bands(x, band))
    res.extend(ee.List([x for x in tmp]).getInfo())

  return res

def process_hila(df):
  bands = ['B2', 'B3', 'B4', 'B8']

  with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    futures = [executor.submit(extract_bands, df['coords'], band) for band in bands]

  for i, future in enumerate(futures):
    band = bands[i]
    df[[f'{band}_mean',f'{band}_std', f'{band}_max', f'{band}_min']] = future.result()
  
  return df

def conv_Polygon(poly):
  x,y = poly.exterior.coords.xy
  coords = np.dstack((x,y)).tolist()
  #geo = ee.Geometry.Polygon(coords)
  #feature = ee.Feature(geo)
  return coords

Coordinates here decide the area, arbitrary Polygon shapes are technically supported.
The resolution is forced 16x16 meters from the Hila-data, so predictions on smaller areas, especially if not easily dividable by 16 can be inaccurate.

In [ ]:
coords = [[24.3218, 61.2715], 
     [24.3218, 61.2627], 
     [24.3405, 61.2627], 
     [24.3405, 61.2715]]

hila = extract_hila(coords)
hila['coords'] = hila['geometry'].map(conv_Polygon)
hila = process_hila(hila)

In [ ]:
def set_index_variables(df):    
    df['ndvi'] = (df['B8_mean'] - df['B4_mean'])/(df['B8_mean'] + df['B4_mean'])
    df['gndvi'] = (df['B8_mean'] - df['B3_mean'])/(df['B8_mean'] + df['B3_mean'])
    df['evi'] = 2.5 * ((df['B8_mean'] - df['B4_mean'])/(df['B8_mean'] - 6*df['B4_mean'] - 7.5*df['B2_mean'] + 1))
    df['sr'] = df['B8_mean'] / df['B4_mean']
    df['msr'] = ((df['B8_mean'])/(df['B4_mean']-1)) / (np.sqrt((df['B8_mean'])/(df['B4_mean']))+1)
    df['savi'] = (1+1) * (df['B8_mean']-df['B4_mean'])/(df['B8_mean']+df['B4_mean'])
    df['ctvi'] = (df['ndvi']+0.5)/(abs(df['ndvi']+0.5)) * np.sqrt(abs(df['ndvi']+0.5))
    df['ttvi'] = np.sqrt(abs((df['B8_mean']-df['B4_mean'])/(df['B8_mean']+df['B4_mean']) + 0.5))
    df['rvi'] = df['B4_mean'] / df['B8_mean']
    df['nrvi'] = (df['rvi']-1)/(df['rvi']+1)
    df['ipvi'] = (df['B8_mean']) / (df['B8_mean']+df['B4_mean'])
    df['osavi'] = (df['B8_mean']-df['B4_mean']) / (df['B8_mean']+df['B4_mean']+0.16)
    df['tndvi'] = np.sqrt(df['ndvi']+0.5)
    df['grvi'] = (df['B3_mean']-df['B4_mean']) / (df['B3_mean']+df['B4_mean'])
    df['arvi'] = (df['B8_mean']-(2*df['B4_mean']-df['B2_mean']))/(df['B8_mean']+(2*df['B4_mean']-df['B2_mean']))
    return df

In [ ]:
rf = joblib.load('./rf-final.joblib')
rf

In [ ]:
selected_features = ['B8_max', 'grvi', 'B2_mean', 'B3_mean', 'B8_min', 'B4_std', 'B3_max', 'B8_mean',
                     'gndvi', 'B8_std', 'B2_max', 'B4_min', 'B2_min', 'B4_mean', 'B3_min', 'B2_std',
                     'B4_max', 'msr', 'ctvi', 'rvi', 'osavi', 'sr', 'ndvi', 'nrvi', 'ipvi', 'ttvi', 
                     'savi', 'tndvi', 'evi', 'B3_std']

X = hila[selected_features]
y_pred = rf.predict(X)
y_pred